In [1]:
import pandas as pd
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
import spacy
from ast import literal_eval
import folium
import re


df1 = pd.read_csv('April 16 - June 12, 2021 T42 Screening Form Submissions.csv')
df2 = pd.read_csv('June 15 - August 11, 2021 T42 Screening Form Submissions.csv')
df3 = pd.read_csv("August 2021 T42 Screening.csv")

df = pd.concat([df1, df2, df3], axis=0)
df

,Date,First Name,Middle Name,Last Name,Family Size,DOB,Citizenship Country,Birth Country,Birth City,Gender,...,August Referral,Prior Deportation,Explain Prior Deportation,Status in MX,Explain Status in MX,UniqueFamilyID,Family Size Formatting,UniqueFamilyIDFormatted,August 2021 Referral,HP + T42 Exemption to CBP
0,NaN,Julio,Felipe,Cuá chicol,2,4/11/1994,Guatemala,Guatemala,Guatemala departamento Chimaltenango municipio...,Hombre,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Carlos,Alexis,Cuá Esquit,2,9/1/2017,Guatemala,Guatemala,"Guatemala, Chimaltenango",Hombre,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Amado,Reniery,Velásquez sequeiro,7,12/7/1990,Honduras,Honduras,La ceiba,Hombre,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Denia,Aracely,Soto marroquin,7,6/10/1987,Honduras,Honduras,La ceiba,Mujer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Celina,Beatriz,Velásquez Soto,7,26/12/2007,Honduras,Honduras,La ceiba,Mujer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,8/24/2021,María,Fernanda,Martinez Barraza,1 - Viajo con una persona mas.,3/2/2007,Mexico,Mexico,Juarez chihuahua,Mujer,...,NaN,No,NaN,No,NaN,1931Hernández 15,1,1931Hernández15,checked,NaN
52,8/25/2021,Sonia,Etelinda,Sorto Molina,Yo viajo solo/a.,26/3/1976,El Salvador,El Salvador,San Miguel,Mujer,...,NaN,No,NaN,Si,residencia permanente en México,2634Sanchez Sorto03,Yo,2634SanchezSorto03,checked,NaN
53,8/26/2021,Victor,Javier,Ordoñez lagos,Yo viajo solo/a.,23/3/1997,Honduras,Honduras,Trojes el paraíso,No especificado o Transgénero,...,NaN,No,NaN,No,NaN,6757Funez22,Yo,6757Funez22,checked,NaN
54,8/26/2021,SARA,GUADALUPE,MIRANDA,3 - Viajo con tres personas mas.,30/3/1988,El Salvador,El Salvador,La paz,Mujer,...,NaN,No sé,NaN,No,NaN,2493Urbina01,3,2493Urbina01,checked,NaN


In [2]:
df['Language'].value_counts()

Language
Español     1489
Kekchi        23
Quiche        15
Chuj           7
Mam            6
English        5
Kanjobal       4
Soltero        4
Kreyol         3
Mixteco        2
Otro           1
Qeqchi         1
Name: count, dtype: int64

In [23]:
non_indigenous_languages = ["Español", "English","Kreyol"]

# Count the number of rows that don't have the specific value in the column
count = df["Language"].replace(non_indigenous_languages, None).value_counts().sum()
print(f"Percentage of Indigenous Language Speaking Parties - \033[1m{(count/len(df['Language'].dropna())):.2%}\033[0m")


Percentage of Indigenous Language Speaking Parties - 4.04%


In [4]:
df["Prior Deportation"].value_counts()

Prior Deportation
No       545
No sé     27
Si        22
Name: count, dtype: int64

In [9]:
replace_values = {"No he entrado a los Estados Unidos durante el último año.":"0",
                 "Más de 5": "5"
                 }
df["# Returned to MX"] = df["# Returned to MX"].replace(replace_values)
df["# Returned to MX"] = pd.to_numeric(df["# Returned to MX"], errors='coerce')
df["# Returned to MX"] = df["# Returned to MX"].fillna(-1).astype(int)

df["# Returned to MX"].value_counts()

# Returned to MX
 1     480
-1     461
 0     353
 2     168
 3      49
 4      30
 5      12
 6       5
 18      4
 11      2
Name: count, dtype: int64

In [ ]:
df["Hair Color"].value_counts()

In [12]:
df["Only US Entry Date"].isna().sum()

1283

In [13]:
df["First US Entry Date"].isna().sum()

1401

In [14]:
df["Mexico Entry Date"].isna().sum()

822

In [16]:
df["Date of Leaving Mexico"] = df['First US Entry Date'].fillna(df['Only US Entry Date'])
df["Date of Leaving Mexico"].isna().sum()

1120

In [27]:
df['Date of Leaving Mexico'] = pd.to_datetime(df['Date of Leaving Mexico'], format='%d/%m/%Y')
df['Mexico Entry Date'] = pd.to_datetime(df['Mexico Entry Date'], format='%d/%m/%Y')
df["travel_time"] = df['Date of Leaving Mexico'] - df['Mexico Entry Date']

print(f"Average time through Mexico from date entered to day of first attempt at crossing: \033[1m {df['travel_time'].mean().days} days \033[0m")


Average time through Mexico from date entered to day of first attempt at crossing:  77 days 
